In [1]:
%load_ext autoreload
%autoreload 2

# exploration of task 1

## exploration of directory structure


In [3]:
bucket="gs://dsgt-clef-erisk-2024"
! gcloud storage ls {bucket}

gs://dsgt-clef-erisk-2024/task1/
gs://dsgt-clef-erisk-2024/task2/


In [4]:
! gcloud storage ls {bucket}/task1

gs://dsgt-clef-erisk-2024/task1/dataset/
gs://dsgt-clef-erisk-2024/task1/training/


In [18]:
! gcloud storage du --readable-sizes {bucket}/task1/dataset | head -n 5
! gcloud storage du --readable-sizes {bucket}/task1/dataset | tail -n 5
! gcloud storage ls {bucket}/task1/dataset | wc -l

3.54kiB      gs://dsgt-clef-erisk-2024/task1/dataset/s_0.trec
10.73MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_1.trec
10.65MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_10.trec
10.95MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_100.trec
10.77MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_101.trec
11.15MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_98.trec
10.62MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_99.trec
839.27MiB    gs://dsgt-clef-erisk-2024/task1/dataset/t1_test_collection_2024.zip
2.53GiB      gs://dsgt-clef-erisk-2024/task1/dataset/test.parquet
9.35GiB      gs://dsgt-clef-erisk-2024/task1/dataset/
555


In [10]:
! gcloud storage du --readable-sizes {bucket}/task1/training

gs://dsgt-clef-erisk-2024/task1/training/training.parquet
gs://dsgt-clef-erisk-2024/task1/training/t1_training/


In [21]:
! gcloud storage du --readable-sizes "{bucket}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/" | head -n 5
! gcloud storage du --readable-sizes "{bucket}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/" | tail -n 5
! gcloud storage du "{bucket}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/" | wc -l

406.81kiB    gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_qrels_majority_2.csv
406.81kiB    gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_rels_consenso.csv
22.86kiB     gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_0.trec
44.65kiB     gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_1.trec
200.45kiB    gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_10.trec
305.75kiB    gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_997.trec
2.09kiB      gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_998.trec
190.46kiB    gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_999.trec
745.75MiB    gs://dsgt-clef-erisk-2024/task1/training/t1_t

## exploration of data

- https://www.ismanet.org/doctoryourspirit/pdfs/Beck-Depression-Inventory-BDI.pdf


In [4]:
from erisk.utils import get_spark

bucket = "gs://dsgt-clef-erisk-2024"
spark = get_spark()
spark

In [5]:
test = spark.read.parquet(f"{bucket}/task1/dataset/test.parquet")
test.printSchema()

root
 |-- DOCNO: string (nullable = true)
 |-- PRE: string (nullable = true)
 |-- TEXT: string (nullable = true)
 |-- POST: string (nullable = true)



In [7]:
test.show(n=5, vertical=True, truncate=80)

-RECORD 0---------------------------------------------------------------------------------
 DOCNO | 0_0_0                                                                            
 PRE   | NULL                                                                             
 TEXT  | I guess it depends on what cheating entails.                                     
 POST  | I met someone who understands me better than I understand myself.                
-RECORD 1---------------------------------------------------------------------------------
 DOCNO | 0_0_1                                                                            
 PRE   | NULL                                                                             
 TEXT  | I met someone who understands me better than I understand myself.                
 POST  | Our friendship thus far is completely platonic, but I know if I had the oppor... 
-RECORD 2---------------------------------------------------------------------------------

In [18]:
# rels consensus
rels_consensus = spark.read.csv(
    f"{bucket}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_rels_consenso.csv",
    header=True,
    inferSchema=True,
)
rels_consensus.printSchema()
rels_consensus.show(n=5, truncate=80)
rels_consensus.count()

root
 |-- query: integer (nullable = true)
 |-- q0: integer (nullable = true)
 |-- docid: string (nullable = true)
 |-- rel: integer (nullable = true)

+-----+---+-------------+---+
|query| q0|        docid|rel|
+-----+---+-------------+---+
|    1|  0|s_405_1279_15|  1|
|    1|  0| s_2519_356_0|  0|
|    1|  0|  s_2038_51_7|  1|
|    1|  0|   s_975_61_2|  0|
|    1|  0|  s_577_923_1|  1|
+-----+---+-------------+---+
only showing top 5 rows



21580

In [19]:
rels_majority = spark.read.csv(
    f"{bucket}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_qrels_majority_2.csv",
    header=True,
    inferSchema=True,
)
rels_majority.printSchema()
rels_majority.show(n=5, truncate=80)
rels_majority.count()

root
 |-- query: integer (nullable = true)
 |-- q0: integer (nullable = true)
 |-- docid: string (nullable = true)
 |-- rel: integer (nullable = true)

+-----+---+-------------+---+
|query| q0|        docid|rel|
+-----+---+-------------+---+
|    1|  0|s_405_1279_15|  1|
|    1|  0| s_2519_356_0|  0|
|    1|  0|  s_2038_51_7|  1|
|    1|  0|   s_975_61_2|  1|
|    1|  0|  s_577_923_1|  1|
+-----+---+-------------+---+
only showing top 5 rows



21580